In [1]:
from sklearn import datasets

In [2]:
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split

In [3]:
import numpy as np
import pandas as pd
df = pd.read_csv('../Data/global-2-1.csv')

In [4]:
df.shape[1] #nr of columns

32

In [5]:
df.shape[0] #nr of entries

3200

In [6]:
X = df.drop(['MAP_ID', 'RISK_ZONE_ID'], axis=1).values

In [7]:
y = df.loc[:,'RISK_ZONE_ID'].values

In [8]:
y

array([1, 2, 1, ..., 3, 1, 1])

In [9]:
#split intro train and test data
X_train, X_test, y_train, y_test = train_test_split(X,y,random_state=0)

In [10]:
# training a XGBoost classifier
from xgboost import XGBClassifier

In [11]:
clf = XGBClassifier(random_state=700, early_stopping_rounds=10).fit(X_train, y_train) 

In [12]:
y_pred = clf.predict(X_test)

In [13]:
accuracy = clf.score(X_test, y_test)

In [14]:
accuracy

0.9275

In [15]:
cm = confusion_matrix(y_test, y_pred)

In [16]:
cm

array([[287,   1,   0,   0,   0],
       [ 11, 152,   0,   1,   0],
       [  1,  13, 175,  21,   0],
       [  0,   0,   8, 122,   0],
       [  0,   0,   0,   2,   6]])

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
df['risk_zone_nb']=df['RISK_ZONE_ID'].factorize()[0]
risk_zone_nb_df = df[['RISK_ZONE_ID','risk_zone_nb']].drop_duplicates().sort_values('risk_zone_nb')

nb_to_riskzone = dict(risk_zone_nb_df[['risk_zone_nb', 'RISK_ZONE_ID']]).values

In [ ]:
fig, ax = plt.subplots(figsize=(10,10))
sns.heatmap(cm, annot=True, fmt='d', xticklabels=risk_zone_nb_df.RISK_ZONE_ID.values, yticklabels=risk_zone_nb_df.RISK_ZONE_ID.values)
plt.ylabel('Actual')
plt.xlabel('Predicted')
plt.show()

In [ ]:
from sklearn import metrics
print(metrics.classification_report(y_test, y_pred))

### support is the number of actual occurences of the class in the test data set. Imbalanced support in the training data may indicate the need for stratified sampling or rebalancing.
micro-average is preferable if there is a class imbalance problem. The micro-average is used in unbalanced datasets as this method takes the frequency of each class into consideration.
The micro-avg precision, recall and f1-score are mathematically equivalent

In [ ]:
print('Cohen-Kappa score: ',metrics.cohen_kappa_score(y_test, y_pred))